# SW-2-RDFBasics

**Navigation** : [<< 1-Setup](SW-1-Setup.ipynb) | [Index](README.md) | [3-GraphOperations >>](SW-3-GraphOperations.ipynb)

## Les fondamentaux de RDF : Triplets, Noeuds et Serialisation

Ce notebook explore en detail le modele de donnees RDF (Resource Description Framework), la brique fondamentale du Web semantique. Nous etudierons l'anatomie des triplets, les differents types de noeuds, la gestion des espaces de noms et les principaux formats de serialisation.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Decrire l'anatomie d'un triplet RDF (sujet, predicat, objet) et sa signification
2. Distinguer et creer les trois types de noeuds : URI, Blank et Literal
3. Configurer des espaces de noms et des prefixes avec `NamespaceMapper`
4. Serialiser un graphe RDF dans les 4 formats principaux (NTriples, Turtle, RDF/XML, JSON-LD)

### Concepts cles

| Concept | Description |
|---------|-------------|
| Triplet RDF | Unite fondamentale : assertion (sujet, predicat, objet) |
| IUriNode | Noeud identifie par un URI, represente une ressource |
| IBlankNode | Noeud anonyme sans identifiant global |
| ILiteralNode | Noeud contenant une valeur (texte, nombre, date) |
| Espace de noms | Prefixe qui abrege les URIs longues |
| Serialisation | Encodage d'un graphe RDF dans un format textuel |

### Prerequis
- Notebook SW-1-Setup complete

### Duree estimee : 45 minutes

---

## 0. Installation de dotNetRDF

Comme dans le notebook precedent, nous commencons par charger la bibliotheque dotNetRDF et ses espaces de noms principaux.

In [ ]:
// Installation du package dotNetRDF (version epinglee pour reproductibilite)
#r "nuget: dotNetRDF, 3.2.1"

using VDS.RDF;
using VDS.RDF.Parsing;
using VDS.RDF.Writing;
using System;
using System.IO;
using StringWriter = System.IO.StringWriter;

Console.WriteLine("dotNetRDF 3.2.1 charge avec succes.");
Console.WriteLine("Espaces de noms disponibles : VDS.RDF, VDS.RDF.Parsing, VDS.RDF.Writing");

### Interpretation

La cellule ci-dessus installe et importe les trois espaces de noms principaux de dotNetRDF :

| Espace de noms | Contenu | Usage dans ce notebook |
|----------------|---------|------------------------|
| `VDS.RDF` | Classes de base (`Graph`, `Triple`, `INode`) | Creation de graphes et de noeuds |
| `VDS.RDF.Parsing` | Parsers (`TurtleParser`, `RdfXmlParser`) | Lecture de donnees RDF |
| `VDS.RDF.Writing` | Writers (`NTriplesWriter`, `CompressingTurtleWriter`) | Serialisation de graphes |

> **Note technique** : La version `3.2.1` est epinglee pour garantir la reproductibilite. L'alias `using StringWriter = System.IO.StringWriter` evite un conflit de noms avec `VDS.RDF.Writing.StringWriter`.

---

## 1. Les triplets RDF

### Anatomie d'un triplet

En RDF, toute information est exprimee sous forme d'un **triplet** compose de trois elements :

```
  (Sujet)  ---[Predicat]--->  (Objet)
```

| Element | Role | Types autorises |
|---------|------|-----------------|
| **Sujet** | La ressource decrite | URI Node, Blank Node |
| **Predicat** | La relation / propriete | URI Node uniquement |
| **Objet** | La valeur ou la ressource liee | URI Node, Blank Node, Literal Node |

Chaque triplet est une **assertion** qui exprime une relation entre deux entites :

| Phrase naturelle | Sujet | Predicat | Objet |
|------------------|-------|----------|-------|
| "dotNetRDF dit Hello World" | `http://www.dotnetrdf.org` | `http://example.org/says` | `"Hello World"` |
| "Alice connait Bob" | `ex:Alice` | `foaf:knows` | `ex:Bob` |
| "Alice a 30 ans" | `ex:Alice` | `foaf:age` | `"30"^^xsd:integer` |

Un **graphe RDF** est un ensemble de triplets. Visuellement, il forme un graphe oriente :

```
  (Alice) ---connait---> (Bob)
     |                     |
     +---nom---> "Alice"   +---nom---> "Bob"
     |                     |
     +---age---> 30        +---age---> 28
```

Creons nos premiers triplets en C#.

In [ ]:
// Creation d'un graphe vide et de nos premiers triplets
IGraph g = new Graph();

// Creation des noeuds URI (sujet et predicat)
IUriNode dotNetRDF = g.CreateUriNode(UriFactory.Create("http://www.dotnetrdf.org"));
IUriNode says = g.CreateUriNode(UriFactory.Create("http://example.org/says"));

// Creation des noeuds literals (objets)
ILiteralNode helloWorld = g.CreateLiteralNode("Hello World");
ILiteralNode bonjourMonde = g.CreateLiteralNode("Bonjour tout le Monde", "fr");

// Assertion des triplets dans le graphe
g.Assert(new Triple(dotNetRDF, says, helloWorld));
g.Assert(new Triple(dotNetRDF, says, bonjourMonde));

Console.WriteLine($"Nombre de triplets dans le graphe : {g.Triples.Count}");
Console.WriteLine();

// Affichage detaille de chaque triplet
foreach (Triple t in g.Triples)
{
    Console.WriteLine($"  Sujet:    {t.Subject}");
    Console.WriteLine($"  Predicat: {t.Predicate}");
    Console.WriteLine($"  Objet:    {t.Object}");
    Console.WriteLine();
}

### Interpretation : Premier triplet RDF

Nous avons cree un graphe contenant **deux triplets** qui partagent le meme sujet et le meme predicat, mais different par l'objet :

| Triplet | Sujet | Predicat | Objet |
|---------|-------|----------|-------|
| 1 | `<http://www.dotnetrdf.org>` | `<http://example.org/says>` | `"Hello World"` |
| 2 | `<http://www.dotnetrdf.org>` | `<http://example.org/says>` | `"Bonjour tout le Monde"@fr` |

**Points cles** :
1. `g.Assert()` ajoute un triplet au graphe. C'est l'equivalent de "declarer un fait" dans la base de connaissances.
2. `UriFactory.Create()` interne les URIs pour optimiser la memoire et accelerer les comparaisons d'egalite.
3. Le second literal possede une **etiquette de langue** (`"fr"`), ce qui en fait un noeud distinct du premier.

> **Note** : La sortie affichee par les proprietes `Subject`, `Predicate`, `Object` est une representation de debogage, pas une syntaxe RDF standard. Pour une serialisation conforme, il faut utiliser un Writer (cf. section 4).

---

## 2. Types de noeuds

RDF definit trois types de noeuds fondamentaux, representes dans dotNetRDF par des interfaces specifiques.

| Type | Interface dotNetRDF | Notation RDF | Roles autorises |
|------|--------------------|--------------|-----------------|
| **URI Node** | `IUriNode` | `<http://...>` | Sujet, Predicat, Objet |
| **Blank Node** | `IBlankNode` | `_:id` | Sujet, Objet |
| **Literal Node** | `ILiteralNode` | `"valeur"` | Objet uniquement |

### 2.1 URI Nodes (`IUriNode`)

Les noeuds URI identifient de maniere **unique et globale** une ressource sur le Web. Ils constituent le coeur de RDF : chaque ressource, chaque propriete est identifiee par un URI.

Deux methodes de creation existent dans dotNetRDF.

In [ ]:
IGraph g = new Graph();

// Methode 1 : URI absolue avec UriFactory
IUriNode person = g.CreateUriNode(UriFactory.Create("http://example.org/person/Alice"));
Console.WriteLine($"URI absolue : {person.Uri}");

// Methode 2 : URI avec prefixe (QName) - necessite un namespace declare
g.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));
IUriNode knows = g.CreateUriNode("ex:knows");
Console.WriteLine($"URI via prefixe : {knows.Uri}");

// Les deux methodes produisent des IUriNode equivalents
IUriNode alice2 = g.CreateUriNode("ex:person/Alice");
Console.WriteLine($"\nComparaison : {person.Uri} == {alice2.Uri} ? {person.Uri == alice2.Uri}");
Console.WriteLine($"Type du noeud : {person.NodeType}");

### Interpretation : Creation d'URI Nodes

| Methode | Code | Quand l'utiliser |
|---------|------|------------------|
| **URI absolue** | `g.CreateUriNode(UriFactory.Create("..."))` | URIs ponctuelles, URIs externes |
| **Prefixe QName** | `g.CreateUriNode("prefix:local")` | Usage repete d'un meme namespace |

**Points cles** :
1. Les URIs sont des **identifiants globaux** : deux noeuds avec la meme URI sont identiques, meme dans des graphes differents.
2. `UriFactory.Create()` optimise la memoire en **internant** les URIs (une seule instance par URI distincte).
3. La methode QName necessite que le prefixe soit prealablement declare dans `g.NamespaceMap`.

### 2.2 Blank Nodes (`IBlankNode`)

Les **noeuds anonymes** (blank nodes) representent des ressources dont on ne connait pas (ou n'a pas besoin de) l'identifiant global. Ils sont identifies localement par un ID interne au graphe.

Cas d'usage typiques :
- Structures intermediaires (adresses composees, coordonnees)
- Ressources dont l'identite exacte n'importe pas
- Existentielles : "il existe quelque chose qui..."

In [ ]:
IGraph g = new Graph();
g.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));

// Blank node avec ID genere automatiquement
IBlankNode bnode1 = g.CreateBlankNode();
Console.WriteLine($"Blank node auto     : _:{bnode1.InternalID}");

// Blank node avec ID explicite
IBlankNode address = g.CreateBlankNode("addr1");
Console.WriteLine($"Blank node explicite : _:{address.InternalID}");

// Utilisation : Alice a une adresse (structure anonyme)
IUriNode alice = g.CreateUriNode("ex:Alice");
IUriNode hasAddress = g.CreateUriNode("ex:hasAddress");
IUriNode street = g.CreateUriNode("ex:street");
IUriNode city = g.CreateUriNode("ex:city");

g.Assert(new Triple(alice, hasAddress, address));
g.Assert(new Triple(address, street, g.CreateLiteralNode("42 rue de la Paix")));
g.Assert(new Triple(address, city, g.CreateLiteralNode("Paris")));

Console.WriteLine($"\nNombre de triplets : {g.Triples.Count}");
foreach (Triple t in g.Triples)
{
    Console.WriteLine($"  {t}");
}

### Interpretation : Blank Nodes

Le blank node `_:addr1` sert de **pivot** pour regrouper les proprietes de l'adresse :

```
  ex:Alice ---ex:hasAddress---> _:addr1
                                  |--- ex:street ---> "42 rue de la Paix"
                                  |--- ex:city   ---> "Paris"
```

| Propriete | Blank Node vs URI Node |
|-----------|------------------------|
| **Portee** | Locale au graphe (pas d'identifiant global) |
| **Reutilisabilite** | Ne peut pas etre reference depuis un autre graphe |
| **Notation** | `_:id` en NTriples/Turtle |
| **Creation auto** | `g.CreateBlankNode()` genere un ID unique |

> **Bonne pratique** : Preferez les URI Nodes pour les ressources importantes. Reservez les Blank Nodes aux structures intermediaires ou temporaires.

### 2.3 Literal Nodes (`ILiteralNode`)

Les **noeuds literaux** representent des valeurs concretes : texte, nombres, dates, booleens. Ils ne peuvent apparaitre qu'en **position objet** dans un triplet.

Trois variantes existent :

| Variante | Syntaxe RDF | Exemple |
|----------|------------|---------|
| **Plain literal** | `"texte"` | `"Hello World"` |
| **Literal avec langue** | `"texte"@lang` | `"Bonjour"@fr` |
| **Literal type** | `"valeur"^^xsd:type` | `"42"^^xsd:integer` |

In [ ]:
IGraph g = new Graph();

// --- Variante 1 : Plain literal (chaine simple) ---
ILiteralNode plain = g.CreateLiteralNode("Hello World");
Console.WriteLine($"Plain literal   : \"{plain.Value}\"");

// --- Variante 2 : Literal avec etiquette de langue ---
ILiteralNode langFr = g.CreateLiteralNode("Bonjour le monde", "fr");
ILiteralNode langEn = g.CreateLiteralNode("Hello World", "en");
Console.WriteLine($"Literal @fr     : \"{langFr.Value}\"@{langFr.Language}");
Console.WriteLine($"Literal @en     : \"{langEn.Value}\"@{langEn.Language}");

// --- Variante 3 : Literal type (avec type XSD) ---
// Methode manuelle (verbose)
ILiteralNode ageManuel = g.CreateLiteralNode("30",
    UriFactory.Create(XmlSpecsHelper.XmlSchemaDataTypeInteger));
Console.WriteLine($"\nTyped (manuel)  : \"{ageManuel.Value}\"^^<{ageManuel.DataType}>");

// Methode avec extension .ToLiteral() (recommandee)
INode ageExt = (30).ToLiteral(g);
INode piExt = (3.14159).ToLiteral(g);
INode dateExt = DateTime.Now.ToLiteral(g);
INode boolExt = (true).ToLiteral(g);

Console.WriteLine($"\nTyped via .ToLiteral() :");
Console.WriteLine($"  int      -> {ageExt}");
Console.WriteLine($"  double   -> {piExt}");
Console.WriteLine($"  DateTime -> {dateExt}");
Console.WriteLine($"  bool     -> {boolExt}");

### Interpretation : Les trois variantes de literals

| Variante | Code C# | Notation RDF | Type XSD |
|----------|---------|-------------|----------|
| Plain | `CreateLiteralNode("Hello")` | `"Hello"` | `xsd:string` (implicite) |
| Langue | `CreateLiteralNode("Bonjour", "fr")` | `"Bonjour"@fr` | Aucun (mutuellement exclusif) |
| Type int | `(30).ToLiteral(g)` | `"30"^^xsd:integer` | `xsd:integer` |
| Type double | `(3.14).ToLiteral(g)` | `"3.14"^^xsd:double` | `xsd:double` |
| Type date | `DateTime.Now.ToLiteral(g)` | `"2025-..."^^xsd:dateTime` | `xsd:dateTime` |
| Type bool | `(true).ToLiteral(g)` | `"true"^^xsd:boolean` | `xsd:boolean` |

**Points cles** :
1. Les methodes d'extension `.ToLiteral()` evitent la construction manuelle des URIs XSD et rendent le code beaucoup plus lisible.
2. Un literal **ne peut pas** avoir a la fois un type et une etiquette de langue (c'est l'un ou l'autre).
3. Sans type ni langue, un literal est considere comme un `xsd:string` par defaut.

> **Note technique** : Les types XSD (XML Schema Datatypes) definissent un ensemble standard de types de donnees utilises dans tout le Web semantique. Pour les types courants (`int`, `DateTime`, etc.), les extensions `.ToLiteral()` produisent automatiquement l'encodage RDF correct.

### Recapitulatif : les trois types ensemble

Construisons un graphe utilisant les trois types de noeuds en combinaison.

In [ ]:
// Graphe combinant les trois types de noeuds
IGraph g = new Graph();
g.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));
g.NamespaceMap.AddNamespace("foaf", UriFactory.Create("http://xmlns.com/foaf/0.1/"));

// URI Nodes (ressources nommees)
IUriNode alice = g.CreateUriNode("ex:Alice");
IUriNode bob = g.CreateUriNode("ex:Bob");
IUriNode knows = g.CreateUriNode("foaf:knows");
IUriNode name = g.CreateUriNode("foaf:name");
IUriNode age = g.CreateUriNode("foaf:age");
IUriNode rdfType = g.CreateUriNode("rdf:type");
IUriNode foafPerson = g.CreateUriNode("foaf:Person");

// Blank Node (competence anonyme d'Alice)
IBlankNode skill = g.CreateBlankNode("skill1");
IUriNode hasSkill = g.CreateUriNode("ex:hasSkill");
IUriNode skillName = g.CreateUriNode("ex:skillName");
IUriNode skillLevel = g.CreateUriNode("ex:skillLevel");

// Assertions avec les 3 types de noeuds
g.Assert(new Triple(alice, rdfType, foafPerson));                                    // URI -> URI
g.Assert(new Triple(alice, name, g.CreateLiteralNode("Alice Dupont", "fr")));       // URI -> Literal@fr
g.Assert(new Triple(alice, age, (30).ToLiteral(g)));                                 // URI -> Literal^^xsd:int
g.Assert(new Triple(alice, knows, bob));                                              // URI -> URI
g.Assert(new Triple(alice, hasSkill, skill));                                         // URI -> Blank
g.Assert(new Triple(skill, skillName, g.CreateLiteralNode("C#")));                  // Blank -> Literal
g.Assert(new Triple(skill, skillLevel, g.CreateLiteralNode("Expert")));             // Blank -> Literal
g.Assert(new Triple(bob, rdfType, foafPerson));                                      // URI -> URI
g.Assert(new Triple(bob, name, g.CreateLiteralNode("Bob Martin", "fr")));          // URI -> Literal@fr

Console.WriteLine($"Graphe avec {g.Triples.Count} triplets :");
Console.WriteLine();
foreach (Triple t in g.Triples)
{
    string sType = t.Subject is IBlankNode ? "Blank" : "URI";
    string oType = t.Object is IBlankNode ? "Blank" : (t.Object is ILiteralNode ? "Literal" : "URI");
    Console.WriteLine($"  [{sType}] {t.Subject}  -->  {t.Predicate}  -->  [{oType}] {t.Object}");
}

### Interpretation : Graphe multi-types

Ce graphe illustre toutes les combinaisons de types rencontrees en pratique :

| Triplet | Sujet | Predicat | Objet | Pattern |
|---------|-------|----------|-------|---------|
| Alice est une Person | URI | URI | URI | Classification |
| Alice a un nom | URI | URI | Literal@fr | Propriete textuelle multilingue |
| Alice a 30 ans | URI | URI | Literal^^xsd:int | Propriete numerique |
| Alice connait Bob | URI | URI | URI | Relation entre ressources |
| Alice a une competence | URI | URI | Blank | Lien vers structure anonyme |
| La competence s'appelle C# | Blank | URI | Literal | Detail de la structure |

> **Point cle** : Le predicat est **toujours** un URI Node. C'est la seule position contrainte a un seul type.

---

## 3. Espaces de noms et prefixes

### Le probleme des URIs longues

Sans prefixes, chaque URI doit etre ecrite en entier, ce qui rend le code verbeux et difficile a lire :

```csharp
// Sans prefixes (fastidieux)
g.CreateUriNode(UriFactory.Create("http://xmlns.com/foaf/0.1/name"));
g.CreateUriNode(UriFactory.Create("http://xmlns.com/foaf/0.1/knows"));
g.CreateUriNode(UriFactory.Create("http://xmlns.com/foaf/0.1/mbox"));

// Avec prefixes (lisible)
g.CreateUriNode("foaf:name");
g.CreateUriNode("foaf:knows");
g.CreateUriNode("foaf:mbox");
```

Les **espaces de noms** (namespaces) associent un **prefixe court** a un **URI de base**. Le `NamespaceMapper` de dotNetRDF gere cette correspondance.

In [ ]:
IGraph g = new Graph();

// Prefixes predefinis par dotNetRDF
Console.WriteLine("=== Prefixes predefinis ===");
foreach (var prefix in g.NamespaceMap.Prefixes)
{
    Console.WriteLine($"  {prefix}: -> {g.NamespaceMap.GetNamespaceUri(prefix)}");
}

// Ajout de prefixes standard
g.NamespaceMap.AddNamespace("foaf", UriFactory.Create("http://xmlns.com/foaf/0.1/"));
g.NamespaceMap.AddNamespace("dc", UriFactory.Create("http://purl.org/dc/elements/1.1/"));
g.NamespaceMap.AddNamespace("schema", UriFactory.Create("http://schema.org/"));
g.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));

Console.WriteLine("\n=== Apres ajout des prefixes personnalises ===");
foreach (var prefix in g.NamespaceMap.Prefixes)
{
    Console.WriteLine($"  {prefix}: -> {g.NamespaceMap.GetNamespaceUri(prefix)}");
}

// Utilisation des prefixes pour creer des noeuds
IUriNode foafName = g.CreateUriNode("foaf:name");
IUriNode dcTitle = g.CreateUriNode("dc:title");
Console.WriteLine($"\nResolution : foaf:name -> {foafName.Uri}");
Console.WriteLine($"Resolution : dc:title  -> {dcTitle.Uri}");

### Interpretation : NamespaceMapper

**Prefixes predefinis** dans tout graphe dotNetRDF :

| Prefixe | URI | Usage |
|---------|-----|-------|
| `rdf:` | `http://www.w3.org/1999/02/22-rdf-syntax-ns#` | Vocabulaire RDF de base (`type`, `Property`) |
| `rdfs:` | `http://www.w3.org/2000/01/rdf-schema#` | Schema RDF (`Class`, `subClassOf`, `label`) |
| `xsd:` | `http://www.w3.org/2001/XMLSchema#` | Types de donnees (`integer`, `string`, `dateTime`) |

**Prefixes standard recommandes** (a ajouter selon le domaine) :

| Prefixe | URI | Domaine |
|---------|-----|--------|
| `foaf:` | `http://xmlns.com/foaf/0.1/` | Personnes et relations sociales |
| `dc:` | `http://purl.org/dc/elements/1.1/` | Metadonnees Dublin Core |
| `schema:` | `http://schema.org/` | SEO / donnees structurees (Google) |
| `skos:` | `http://www.w3.org/2004/02/skos/core#` | Thesaurus et taxonomies |
| `owl:` | `http://www.w3.org/2002/07/owl#` | Ontologies Web |

> **Bonne pratique** : Declarez vos prefixes en debut de notebook pour eviter les URIs longues dans le code. Le registre officiel des prefixes est disponible sur [prefix.cc](http://prefix.cc).

### Resolution et reduction d'URIs

Le `NamespaceMapper` fonctionne dans les deux sens : resolution (prefixe vers URI complete) et reduction (URI complete vers prefixe).

In [ ]:
IGraph g = new Graph();
g.NamespaceMap.AddNamespace("foaf", UriFactory.Create("http://xmlns.com/foaf/0.1/"));

// Resolution : prefixe -> URI complete
Uri fullUri = g.NamespaceMap.GetNamespaceUri("foaf");
Console.WriteLine($"Resolution : foaf: -> {fullUri}");

// Reduction : URI complete -> prefixe:local
Uri testUri = UriFactory.Create("http://xmlns.com/foaf/0.1/name");
string reduced;
if (g.NamespaceMap.ReduceToQName(testUri.ToString(), out reduced))
{
    Console.WriteLine($"Reduction  : {testUri} -> {reduced}");
}

// Verification d'existence de prefixes
Console.WriteLine($"\nPrefixe 'foaf' existe ? {g.NamespaceMap.HasNamespace("foaf")}");
Console.WriteLine($"Prefixe 'owl' existe ?  {g.NamespaceMap.HasNamespace("owl")}");

### Interpretation : Operations sur les namespaces

| Operation | Methode | Direction |
|-----------|---------|----------|
| **Ajout** | `AddNamespace(prefix, uri)` | Declaration d'un prefixe |
| **Resolution** | `GetNamespaceUri(prefix)` | Prefixe -> URI complete |
| **Reduction** | `ReduceToQName(uri, out qname)` | URI complete -> Prefixe:local |
| **Verification** | `HasNamespace(prefix)` | Le prefixe existe-t-il ? |

> **Note technique** : Les prefixes sont essentiels pour la lisibilite en Turtle et en SPARQL. Les Writers de dotNetRDF les utilisent automatiquement lors de la serialisation.

---

## 4. Formats de serialisation

Un graphe RDF est une structure abstraite (un ensemble de triplets). Pour le stocker ou le transmettre, il faut le **serialiser** dans un format textuel. Quatre formats principaux existent, chacun avec ses forces et faiblesses.

Nous allons creer un graphe de reference, puis le serialiser dans chaque format pour comparer.

In [ ]:
// Graphe de reference pour les demonstrations de serialisation
IGraph demo = new Graph();
demo.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));
demo.NamespaceMap.AddNamespace("foaf", UriFactory.Create("http://xmlns.com/foaf/0.1/"));

IUriNode alice = demo.CreateUriNode("ex:Alice");
IUriNode bob = demo.CreateUriNode("ex:Bob");
IUriNode rdfType = demo.CreateUriNode("rdf:type");
IUriNode foafPerson = demo.CreateUriNode("foaf:Person");
IUriNode foafName = demo.CreateUriNode("foaf:name");
IUriNode foafAge = demo.CreateUriNode("foaf:age");
IUriNode foafKnows = demo.CreateUriNode("foaf:knows");

demo.Assert(new Triple(alice, rdfType, foafPerson));
demo.Assert(new Triple(alice, foafName, demo.CreateLiteralNode("Alice Dupont", "fr")));
demo.Assert(new Triple(alice, foafAge, (30).ToLiteral(demo)));
demo.Assert(new Triple(alice, foafKnows, bob));
demo.Assert(new Triple(bob, rdfType, foafPerson));
demo.Assert(new Triple(bob, foafName, demo.CreateLiteralNode("Bob Martin", "fr")));

Console.WriteLine($"Graphe de reference cree : {demo.Triples.Count} triplets.");
Console.WriteLine("Ce graphe sera serialise dans 4 formats differents.");

### 4.1 NTriples : le format le plus simple

**NTriples** est un format ligne par ligne, sans abreviations ni prefixes. Chaque ligne contient un triplet complet, termine par un point.

In [ ]:
// Serialisation en NTriples
var ntWriter = new NTriplesWriter();
var sw = new StringWriter();
ntWriter.Save(demo, sw);

string ntOutput = sw.ToString();
Console.WriteLine("=== Format NTriples ===");
Console.WriteLine(ntOutput);
Console.WriteLine($"Taille : {ntOutput.Length} caracteres");

### Interpretation : Format NTriples

**Conventions syntaxiques** :
- URIs toujours entre chevrons : `<http://example.org/Alice>`
- Literals entre guillemets : `"Alice Dupont"`
- Etiquettes de langue apres le literal : `"Bonjour"@fr`
- Types indiques par `^^` : `"30"^^<http://www.w3.org/2001/XMLSchema#integer>`
- Chaque triplet termine par un point `.`

| Avantage | Inconvenient |
|----------|--------------|
| Tres simple a parser | Tres verbeux (URIs repetees integralement) |
| Pas d'ambiguite | Fichiers volumineux |
| Ideal pour le streaming ligne par ligne | Peu lisible pour un humain |

### 4.2 Turtle : le format lisible

**Turtle** (Terse RDF Triple Language) est le format le plus lisible pour les humains. Il utilise des prefixes, des abreviations et des regroupements par sujet.

In [ ]:
// Serialisation en Turtle (CompressingTurtleWriter pour les abreviations maximales)
var turtleWriter = new CompressingTurtleWriter();
var sw = new StringWriter();
turtleWriter.Save(demo, sw);

string turtleOutput = sw.ToString();
Console.WriteLine("=== Format Turtle ===");
Console.WriteLine(turtleOutput);
Console.WriteLine($"Taille : {turtleOutput.Length} caracteres");

### Interpretation : Format Turtle

Turtle introduit plusieurs **abreviations syntaxiques** :

| Syntaxe | Signification | Exemple |
|---------|---------------|---------|
| `@prefix p: <uri> .` | Declaration de prefixe | `@prefix foaf: <http://xmlns.com/foaf/0.1/> .` |
| `;` | Meme sujet, predicat different | `ex:Alice foaf:name "Alice" ; foaf:age 30 .` |
| `,` | Meme sujet et predicat, objet different | `ex:Alice foaf:knows ex:Bob, ex:Carol .` |
| `a` | Raccourci pour `rdf:type` | `ex:Alice a foaf:Person .` |
| `.` | Fin de la description du sujet | Termine le bloc |

| Avantage | Inconvenient |
|----------|--------------|
| Tres lisible et compact | Parsing plus complexe que NTriples |
| Syntaxe proche de SPARQL | Necessite la declaration des prefixes |
| Format prefere pour la documentation | Moins adapte au streaming |

> **Bonne pratique** : Utilisez `CompressingTurtleWriter` plutot que `TurtleWriter` pour beneficier de toutes les compressions syntaxiques (`a`, `;`, `,`).

### 4.3 RDF/XML : le format historique

**RDF/XML** est le premier format de serialisation RDF, standardise par le W3C en 1999. Il utilise la syntaxe XML avec des espaces de noms.

In [ ]:
// Serialisation en RDF/XML
var rdfXmlWriter = new RdfXmlWriter();
var sw = new StringWriter();
rdfXmlWriter.Save(demo, sw);

string rdfXmlOutput = sw.ToString();
Console.WriteLine("=== Format RDF/XML ===");
Console.WriteLine(rdfXmlOutput);
Console.WriteLine($"Taille : {rdfXmlOutput.Length} caracteres");

### Interpretation : Format RDF/XML

| Element XML | Signification RDF |
|-------------|-------------------|
| `<rdf:RDF>` | Racine du document RDF |
| `<rdf:Description rdf:about="...">` | Declaration d'un sujet |
| `<foaf:name>Alice</foaf:name>` | Triplet (sujet, foaf:name, "Alice") |
| `xmlns:foaf="..."` | Declaration d'espace de noms |

| Avantage | Inconvenient |
|----------|--------------|
| Standard W3C historique | Verbeux (syntaxe XML) |
| Compatible outils XML (XSLT, XPath) | Peu lisible |
| Bon support dans les frameworks | Plusieurs serialisations possibles pour le meme graphe |

> **Note** : Le writer genere automatiquement des prefixes `nsX` pour les URIs non declarees dans le `NamespaceMap`. RDF/XML est de moins en moins utilise au profit de Turtle et JSON-LD.

### 4.4 JSON-LD : le format moderne

**JSON-LD** (JSON for Linked Data) encode des donnees RDF dans du JSON standard. C'est le format privilegie pour les APIs Web et le SEO (Schema.org utilise par Google).

In [ ]:
// Serialisation en JSON-LD
// Note : JsonLdWriter implemente IStoreWriter (pas IRdfWriter)
// Il faut donc envelopper le graphe dans un TripleStore
using VDS.RDF;

var store = new TripleStore();
store.Add(demo);

var jsonLdWriter = new JsonLdWriter();
var sw = new StringWriter();
jsonLdWriter.Save(store, sw);

string jsonLdOutput = sw.ToString();
Console.WriteLine("=== Format JSON-LD ===");
Console.WriteLine(jsonLdOutput);
Console.WriteLine($"Taille : {jsonLdOutput.Length} caracteres");

### Interpretation : Format JSON-LD

JSON-LD utilise des mots-cles speciaux prefixes par `@` :

| Mot-cle | Signification | Equivalent RDF |
|---------|---------------|----------------|
| `@context` | Declaration des prefixes | `@prefix` en Turtle |
| `@id` | Identifiant de la ressource | URI du sujet |
| `@type` | Type de la ressource | `rdf:type` |
| `@value` | Valeur d'un literal | Contenu du literal |
| `@language` | Langue d'un literal | Etiquette de langue |

| Avantage | Inconvenient |
|----------|--------------|
| Compatible ecosysteme JSON / JavaScript | Moins compact que Turtle |
| Utilise par Schema.org et Google | Contextualisation peut etre complexe |
| Facile a integrer dans les APIs REST | Plusieurs formes (expanded, compacted, flattened) |

### Comparaison des 4 formats

| Critere | NTriples | Turtle | RDF/XML | JSON-LD |
|---------|----------|--------|---------|--------|
| **Lisibilite** | Faible | Excellente | Faible | Bonne |
| **Compacite** | Faible | Elevee | Moyenne | Moyenne |
| **Complexite de parsing** | Tres faible | Moyenne | Elevee | Moyenne |
| **Prefixes** | Non | Oui (`@prefix`) | Oui (`xmlns`) | Oui (`@context`) |
| **Streaming** | Oui (ligne par ligne) | Difficile | Difficile | Non |
| **Usage principal** | Dump de donnees | Documentation, SPARQL | Interoperabilite XML | APIs Web, SEO |
| **Writer dotNetRDF** | `NTriplesWriter` | `CompressingTurtleWriter` | `RdfXmlWriter` | `JsonLdWriter` |

> **Recommandation** : Utilisez **Turtle** pour la lisibilite et le travail humain, **NTriples** pour les traitements batch et le streaming, **JSON-LD** pour les APIs Web et le SEO.

---

## 5. Exercice 1 : Modeliser un reseau social

**Objectif** : Creer un graphe RDF modelisant un petit reseau social avec :
- 3 personnes (avec nom, age et email)
- Des relations `foaf:knows` entre elles
- Au moins un Blank Node (par exemple, une adresse)
- Serialiser le resultat en Turtle

**Vocabulaires a utiliser** : `foaf:` (Friend of a Friend), `ex:` (votre domaine)

Le code ci-dessous est une solution possible. Modifiez-le pour decrire votre propre reseau.

In [ ]:
// EXERCICE 1 : Modeliser un reseau social en RDF
IGraph social = new Graph();

// Declaration des prefixes
social.NamespaceMap.AddNamespace("foaf", UriFactory.Create("http://xmlns.com/foaf/0.1/"));
social.NamespaceMap.AddNamespace("ex", UriFactory.Create("http://example.org/"));

// Noeuds predicats reutilisables
IUriNode rdfType = social.CreateUriNode("rdf:type");
IUriNode foafPerson = social.CreateUriNode("foaf:Person");
IUriNode foafName = social.CreateUriNode("foaf:name");
IUriNode foafAge = social.CreateUriNode("foaf:age");
IUriNode foafMbox = social.CreateUriNode("foaf:mbox");
IUriNode foafKnows = social.CreateUriNode("foaf:knows");
IUriNode exAddress = social.CreateUriNode("ex:hasAddress");
IUriNode exStreet = social.CreateUriNode("ex:street");
IUriNode exCity = social.CreateUriNode("ex:city");

// --- Carol ---
IUriNode carol = social.CreateUriNode("ex:Carol");
social.Assert(new Triple(carol, rdfType, foafPerson));
social.Assert(new Triple(carol, foafName, social.CreateLiteralNode("Carol Lefevre", "fr")));
social.Assert(new Triple(carol, foafAge, (28).ToLiteral(social)));
social.Assert(new Triple(carol, foafMbox,
    social.CreateUriNode(UriFactory.Create("mailto:carol@example.org"))));

// --- David ---
IUriNode david = social.CreateUriNode("ex:David");
social.Assert(new Triple(david, rdfType, foafPerson));
social.Assert(new Triple(david, foafName, social.CreateLiteralNode("David Moreau", "fr")));
social.Assert(new Triple(david, foafAge, (35).ToLiteral(social)));
social.Assert(new Triple(david, foafMbox,
    social.CreateUriNode(UriFactory.Create("mailto:david@example.org"))));

// --- Emma ---
IUriNode emma = social.CreateUriNode("ex:Emma");
social.Assert(new Triple(emma, rdfType, foafPerson));
social.Assert(new Triple(emma, foafName, social.CreateLiteralNode("Emma Durand", "fr")));
social.Assert(new Triple(emma, foafAge, (42).ToLiteral(social)));

// Relations foaf:knows
social.Assert(new Triple(carol, foafKnows, david));
social.Assert(new Triple(david, foafKnows, emma));
social.Assert(new Triple(emma, foafKnows, carol));

// Blank Node : adresse de Carol
IBlankNode carolAddr = social.CreateBlankNode("carolAddr");
social.Assert(new Triple(carol, exAddress, carolAddr));
social.Assert(new Triple(carolAddr, exStreet, social.CreateLiteralNode("15 avenue Victor Hugo")));
social.Assert(new Triple(carolAddr, exCity, social.CreateLiteralNode("Lyon")));

// Serialisation en Turtle
var turtleWriter = new CompressingTurtleWriter();
var sw = new StringWriter();
turtleWriter.Save(social, sw);

Console.WriteLine($"Reseau social : {social.Triples.Count} triplets");
Console.WriteLine();
Console.WriteLine(sw.ToString());

### Interpretation de l'exercice 1

Le graphe modelise un reseau social minimal avec les trois types de noeuds :

| Personne | Proprietes | Relations |
|----------|------------|-----------|
| Carol | nom@fr, age^^xsd:int, email (URI), adresse (Blank Node) | connait David |
| David | nom@fr, age^^xsd:int, email (URI) | connait Emma |
| Emma | nom@fr, age^^xsd:int | connait Carol |

**Observations sur la sortie Turtle** :
- Les prefixes `@prefix foaf:` et `@prefix ex:` sont declares en tete
- Les proprietes d'un meme sujet sont regroupees avec `;`
- L'adresse de Carol apparait comme `_:carolAddr` (Blank Node)
- Le raccourci `a` remplace `rdf:type` dans la sortie

---

## 6. Exercice 2 : Comparer les tailles de serialisation

**Objectif** : Serialiser le graphe du reseau social dans les differents formats et comparer les tailles obtenues. Cela illustre concretement les differences de compacite entre formats.

In [ ]:
// EXERCICE 2 : Comparer les tailles de serialisation

// Fonction utilitaire pour serialiser et mesurer
string SerializeGraph(IGraph graph, IRdfWriter writer)
{
    var sw = new StringWriter();
    writer.Save(graph, sw);
    return sw.ToString();
}

// Formats IRdfWriter (NTriples, Turtle, RDF/XML)
var formats = new List<(string Name, IRdfWriter Writer)>
{
    ("NTriples", new NTriplesWriter()),
    ("Turtle", new CompressingTurtleWriter()),
    ("RDF/XML", new RdfXmlWriter())
};

Console.WriteLine($"Graphe de test : {social.Triples.Count} triplets\n");
Console.WriteLine($"{"Format",-12} {"Taille (car)",-15} {"Lignes",-10} {"Ratio vs NT",-15}");
Console.WriteLine(new string('-', 52));

int ntSize = 0;
foreach (var (formatName, writer) in formats)
{
    string output = SerializeGraph(social, writer);
    int size = output.Length;
    int lines = output.Split('\n').Length;

    if (formatName == "NTriples") ntSize = size;
    string ratio = ntSize > 0 ? $"{(double)size / ntSize:P0}" : "---";

    Console.WriteLine($"{formatName,-12} {size,-15} {lines,-10} {ratio,-15}");
}

// JSON-LD : IStoreWriter (necessite un TripleStore)
var jsonStore = new TripleStore();
jsonStore.Add(social);
var jsonLdWriter = new JsonLdWriter();
var jsonSw = new StringWriter();
jsonLdWriter.Save(jsonStore, jsonSw);
string jsonOutput = jsonSw.ToString();
string jsonRatio = ntSize > 0 ? $"{(double)jsonOutput.Length / ntSize:P0}" : "---";
Console.WriteLine($"{"JSON-LD",-12} {jsonOutput.Length,-15} {jsonOutput.Split('\n').Length,-10} {jsonRatio,-15}");

### Interpretation de l'exercice 2

**Resultats attendus** (les valeurs exactes varient selon le graphe) :

| Format | Taille relative | Raison |
|--------|----------------|--------|
| NTriples | 100% (reference) | Aucune compression, URIs completes repetees |
| Turtle | ~40-60% | Prefixes, regroupement par sujet, raccourci `a` |
| RDF/XML | ~70-90% | Espaces de noms XML, structure verbose |
| JSON-LD | ~60-80% | Contexte JSON, structure d'objets |

**Conclusion** : Turtle est presque toujours le format le plus compact grace aux prefixes et aux abreviations syntaxiques. Le choix du format depend de l'usage :
- **Documentation et lecture humaine** : Turtle
- **Echange machine / streaming** : NTriples
- **APIs Web et SEO** : JSON-LD
- **Integration XML** : RDF/XML

---

## Resume

| Element | Ce que nous avons appris |
|---------|-------------------------|
| **Triplet RDF** | Unite fondamentale (sujet, predicat, objet), asserte avec `g.Assert()` |
| **URI Nodes** | `IUriNode` : identifiant global, creation par URI absolue ou QName |
| **Blank Nodes** | `IBlankNode` : noeud anonyme local, pour structures intermediaires |
| **Literal Nodes** | `ILiteralNode` : valeurs (plain, avec langue `@fr`, avec type `^^xsd:`) |
| **Espaces de noms** | `NamespaceMapper` : prefixes pour abreger les URIs (`foaf:`, `dc:`, etc.) |
| **NTriples** | Format simple, une ligne par triplet, pas de prefixes |
| **Turtle** | Format lisible avec prefixes et abreviations (`;`, `,`, `a`) |
| **RDF/XML** | Format historique W3C base sur XML |
| **JSON-LD** | Format moderne pour APIs Web et SEO |

### Ce qui vient ensuite

Dans le notebook suivant (**SW-3-GraphOperations**), nous apprendrons a :
- Selectionner des triplets avec `GetTriplesWithXxx()`
- Fusionner et comparer des graphes
- Lire des fichiers RDF depuis le disque et le Web
- Convertir entre formats de serialisation

---

**Navigation** : [<< 1-Setup](SW-1-Setup.ipynb) | [Index](README.md) | [3-GraphOperations >>](SW-3-GraphOperations.ipynb)